추가로 해볼 수 있는 것

고도화 한다면 빈도수로 하지는 않음
키워드마다 가중치를 준다
- 화제성에 따른 가중치나 다양한 방식 존재  
- 빈도수 이외의 통계 방법 적용 고려

비정형 분석 - text mining
word cloud & table 그리기

1. keyword를 정하고 날짜만 길게 다르게 한다.

2. 10페이지 까지만 가져오기

3. 셋이서 500개 분량 정도

추가적으로 해보고 싶은게 있으면 알려달라고 하심

1. 크롤링으로 제목, 컨텐츠 데이터 확보
2. Cleaning(불용어, 동의어 사전 작성 등)
3. 형태소 분석 통해 명사만 추출 Okt(Open Korean Text)
4. 빈도 분석
5. 시각화

In [31]:
import sys
import os
import pandas as pd
import numpy as np

from selenium import webdriver
import time
import tqdm
from tqdm import tqdm_notebook

In [1]:
# selenium 입력용 변수
query_txt="칼부림"
#
start_date="20230731"
end_date="20200804"

# file 경로와 driver 경로를 같은 곳에 두는 경우 생략 가능
# path = "chromedriver.exe"

driver = webdriver.Chrome()
driver.get('http://www.naver.com')
time.sleep(2)

element = driver.find_element_by_id("query")
element.send_keys(query_txt)
element.submit()
driver.find_element_by_link_text("뉴스").click()

NameError: name 'webdriver' is not defined

In [22]:
titles = "a.news_tit"
article_raw = driver.find_elements_by_css_selector(titles)
article_raw[0].text

'고속터미널역서 ‘흉기 소지’ 남성 체포…살인예고에 “특공대 배치”(종합)'

In [16]:
# overlays = "._sp_each_url" 
overlays = "a.news_tit" 
urls = driver.find_elements_by_css_selector(overlays)
urls[0].get_attribute('href')

'http://www.newsis.com/view/?id=NISX20230807_0002404478&cID=10814&pID=10800'

In [1]:
url_list = []
title_list = []

total_page = 10
for i in tqdm_notebook(range(0, total_page)):
    i = i * 10 + 1
    url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EC%B9%BC%EB%B6%80%EB%A6%BC&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=30&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start={1}1".format(query_txt, i)
    driver.get(url)
    time.sleep(0.5)
    
    
    titles = "a.news_tit"
    articles = "a.news_tit"
    article_raw = driver.find_elements_by_css_selector(titles)
    article_list = driver.find_elements_by_css_selector(articles)
   
    # url 크롤링 시작
    for article in article_raw:
        url = article.get_attribute('href')   
        url_list.append(url)
    
    # 제목 크롤링 시작    
    for article in article_raw:
        title = article.text   
        title_list.append(title)
    
        print(title)
    
print('url갯수: ', len(url_list))

df = pd.DataFrame({'url':url_list, 'title':title_list})

# 저장하기
df.to_csv("news_url.csv")

NameError: name 'tqdm_notebook' is not defined

## 네이버 뉴스 텍스트 분석 

In [25]:
import scipy as sp
import pandas as dp
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline
import os
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
from sklearn.tree import export_graphviz

# 그래프에서 한글 폰트 깨지는 문제(전역 글꼴 설정)
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname = 'C:\Windows\Fonts\malgun.ttf').get_name()
rc('font', family = font_name)

# 워닝 없애주기
import warnings

# 한글 자연어 처리 라이브러리
from konlpy.tag import Kkma
from konlpy.tag import Hannanum
from konlpy.tag import Okt
from konlpy.tag import *
import pickle

kkma = Kkma()
hannanum = Hannanum()
t = Okt()

In [29]:
df_r = pd.read_csv('news_url.csv')
print(df_r['title'])

0               분당 칼부림도, 교사 습격도… ‘치료 거부한 정신질환자’
1        "원주역서 칼부림" 글 쓰고 "제보한다" 퍼뜨린 10대 붙잡혀(종합)
2                 "고양 상가서 칼부림하겠다" 예고글 올린 10대 검거
3         [인터뷰] 범죄학자 "칼부림 막는데 장갑차? 불안만 자극…과유불급"
4        '나도 관심 받을 수 있을까?'…'칼부림 예고'한 10대들 경찰 덜미
                        ...                    
95                        칼부림 예고 글, 보안 강화된 지하철역
96    '살인예고, 올리면 잡힌다'…경찰 "19명 검거·협박 죄 등 처벌"(종합)
97                   [1보]대전 대덕구 고등학교서 칼부림…교사 피습
98              [사설] 누구나 표적되는 ‘묻지마 칼부림’이 일상인 세상
99       "우려가 현실로"…서현 '묻지마 칼부림'에 시민들 '비명' [현장+]
Name: title, Length: 100, dtype: object
